In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-eastus-rg')

In [2]:
import git

# get root of git repo
prefix = git.Repo('.', search_parent_directories=True).working_tree_dir

# training script 
script_dir = prefix+'/code/models/pytorch/mnist-cnn/'
script_name = 'train.py'

# environment file
environment_file = prefix+'/environments/pytorch-example.yml'

# azure ml settings
environment_name = 'pytorch-iris-example'
experiment_name = 'pytorch-mnist-example'
compute_target = 'gpu-cluster'

In [3]:
print(open(script_dir+script_name).read())

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import mlflow
import mlflow.pytorch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(a

In [6]:
from azureml.core import ScriptRunConfig, Experiment, Environment

arguments = ['--epochs', 2]

env = Environment.from_conda_specification(environment_name, environment_file)
env.docker.enabled = True

src = ScriptRunConfig(source_directory=script_dir, script=script_name, arguments=arguments)
src.run_config.environment = env
src.run_config.target = compute_target

run = Experiment(ws, experiment_name).submit(src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
pytorch-mnist-example,pytorch-mnist-example_1598903102_fef081a2,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [ ]:
run.wait_for_completion(show_output=True)